<a href="https://colab.research.google.com/github/aminayusif/Weather-Crop-Advisor/blob/main/Weather_Crop_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

Agricultural productivity is highly influenced by environmental conditions such as temperature, rainfall, soil properties, and humidity. Farmers, especially in developing regions, often face challenges in selecting the most suitable crops for planting due to unpredictable weather patterns and limited access to expert guidance. This can lead to poor yields, wasted resources, and economic losses.

The **Weather-Based Crop Advisor** project aims to support farmers in making informed decisions by using machine learning to recommend the best crops to plant based on prevailing agro-meteorological and soil conditions. By analyzing features like temperature, rainfall, humidity, and soil pH, the system learns patterns from historical data to predict the most suitable crops for a given set of conditions.

This project simulates or integrates environmental datasets to train and evaluate classification models (e.g., decision trees, random forests, or logistic regression) for crop recommendation. The ultimate goal is to provide data-driven insights that empower farmers to optimize yield, reduce risk, and adapt more effectively to climate variability.

By leveraging machine learning in agriculture, this project demonstrates how data science can contribute to sustainable farming, food security, and rural development.

## Objectives

### 1. Crop Recommendation using Classification

Predict the most suitable crop (crop_type) to grow based on weather and soil conditions using classification algorithms like Random Forest, Decision Tree, Support Vector Machine (SVM), K-Nearest Neighbors (KNN)

### 2. Yield Prediction using Regression

Predict crop yield (in kg/hectare or tons) based on environmental factors by using algorithms like Random Forest Regressor, Linear Regression, Gradient Boosting (e.g., XGBoost), Support Vector Regression (SVR).

### 3. Time Series Forecasting

Forecast future rainfall/temperature to assist crop decision using ARIMA or Prophet (by Facebook) for weather trends.

## Dataset Overview

 The dataset for this case study is provided in the file

**simulated_crop_weather_data.csv** and contains approximately 2,000 records. The key attributes include:

 | Column          | Description                            |
| --------------- | -------------------------------------- |
| `id`            | Unique identifier                      |
| `date`          | Observation date                       |
| `region`        | Geographic area within Ghana where the crop is grown
| `temperature`   | Temperature in °C                      |
| `rainfall`      | Rainfall in mm                         |
| `humidity`      | Humidity percentage                    |
| `soil_moisture` | Soil moisture percentage               |
| `soil_type`     | Soil type (e.g., Sandy, Clay, Loamy)          |
| `crop_type`     | Crop grown **(target variable)**           |
| `crop_variety`     | Specific variety of the crop (e.g., Obatanpa maize)**           |
| `yield_kg`      | Yield in kilograms **(regression target)** |
| `soil_pH`      |  Represents the acidity or alkalinity of the soil (can be a key feature in predicting crop suitability)                        |
| `fertilizer_used` | Type of fertilizer applied (e.g., NPK, Urea, Organic, None)                   |
| `fertilizer_quantity_kg` | Quantity of fertilizer applied per hectare               |
| `pesticide_used`     | Whether pesticide was applied (Yes/No)          |
| `planting_method`     |  Direct seeding, transplanting, broadcasting, etc.         |
| `elevation_m`      | Elevation of the farm above sea level (m) |
| `season`     |  This column represents the agricultural season in which the crop was planted or harvested        |
| `irrigation`      | Whether irrigation was used (Yes/No) |
| `planting_density`     | Plants per square meter or hectare          |
| `harvest_method`      |  This indicates the technique used to harvest the crop (Manual or Mechanized) |
| `market_access_km`     |   Represents the distance (in kilometers) from the farm to the nearest market         |
| `extension_services`      |  Indicates whether the farmer had access to agricultural extension services |
      |


## Data Exploration and Preprocessing

### Load and inspect dataset

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler